In [0]:
LIST '/Volumes/databricks_simulated_e_commerce_clickstream_data/v01/raw/events-kafka/'

In [0]:
SELECT * FROM read_files('/Volumes/databricks_simulated_e_commerce_clickstream_data/v01/raw/events-kafka/', format => 'json')

In [0]:
CREATE TABLE kafka_events_bronze AS SELECT * FROM read_files('/Volumes/databricks_simulated_e_commerce_clickstream_data/v01/raw/events-kafka/', format => 'json')

In [0]:
SELECT key, unbase64(key) as decodedKey, value, unbase64(value) decodedValue FROM kafka_events_bronze

In [0]:
CREATE TABLE kafka_events_bronze_decoded AS
SELECT key, cast(unbase64(key) as String) as decodedKey, value, cast(unbase64(value) as String) decodedValue FROM kafka_events_bronze
LIMIT 10;

In [0]:
SELECT decodedValue, 
decodedValue:device,
decodedValue:items,
decodedValue:traffic_source,
decodedValue:geo
FROM kafka_events_bronze_decoded

In [0]:
SELECT schema_of_json('{"device":"Android","ecommerce":{},"event_name":"cc_info","event_previous_timestamp":1593880364321088,"event_timestamp":1593880886065125,"geo":{"city":"New York","state":"NY"},"items":[{"item_id":"M_STAN_F","item_name":"Standard Full Mattress","item_revenue_in_usd":945.0,"price_in_usd":945.0,"quantity":1}],"traffic_source":"instagram","user_first_touch_timestamp":1593879195262047,"user_id":"UA000000107382233"}') as Schema

In [0]:
Create or Replace Table kafka_events_bronze_struct as
SELECT * EXCEPT(decodedValue), 
 from_json(decodedValue, 'STRUCT<device: STRING, event_name: STRING, event_previous_timestamp: BIGINT, event_timestamp: BIGINT, geo: STRUCT<city: STRING, state: STRING>, items: ARRAY<STRUCT<item_id: STRING, item_name: STRING, item_revenue_in_usd: DOUBLE, price_in_usd: DOUBLE, quantity: BIGINT>>, traffic_source: STRING, user_first_touch_timestamp: BIGINT, user_id: STRING>') As StructValue
FROM kafka_events_bronze_decoded

In [0]:
select * from kafka_events_bronze_struct

In [0]:
select StructValue.device,
StructValue.geo,
StructValue.items,
array_size(items) as Itemcount 
from kafka_events_bronze_struct

In [0]:
create or replace table kafka_events_bronze_arrays
as
select decodedKey,
  array_size(structValue.items) itemcount,
  explode(structValue.items) as valuearray,
  StructValue.items
from kafka_events_bronze_struct

In [0]:
select * from kafka_events_bronze_arrays